In [ ]:
#get the probabilities

In [13]:
import pandas as pd
import os
import re
from nltk.stem.porter import *
import math

In [14]:
data = pd.read_csv("Reviews_8000.csv")
numberOfReviews = 8000

In [15]:
def getNgrams(text, n):
    if n == 1:
        return text
    
    text.append("<end>")
    ngramList = []
    for i in range(len(text) - (n-1)):
        ngram = text[i:i+n]
        ngramList.append(ngram)
    
    return ngramList

In [16]:
unigramDict = {}
bigramDict = {}
stemmer = PorterStemmer()

def getCountsinRow(row):
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
    unigram = getNgrams(parsedText, 1)
    bigram = getNgrams(parsedText, 2)
    ngrams = (unigram,bigram)
    for index in range(len(ngrams)):
        for eachWord in ngrams[index]:
            
            if eachWord != "br":
                if index == 0:
                    #unigram

                    if eachWord in unigramDict:
                        unigramDict[eachWord] +=1
                    else:
                        unigramDict[eachWord] =1

                if index == 1:
                    #bigram
                        eachWord = tuple(eachWord)
                        if eachWord in bigramDict:
                            bigramDict[eachWord] +=1
                        else:
                            bigramDict[eachWord] =1

            

In [17]:

#get the probabilies into unigramsDict and bigramsDict
data.apply(getCountsinRow, axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7970    None
7971    None
7972    None
7973    None
7974    None
7975    None
7976    None
7977    None
7978    None
7979    None
7980    None
7981    None
7982    None
7983    None
7984    None
7985    None
7986    None
7987    None
7988    None
7989    None
7990    None
7991    None
7992    None
7993    None
7994    None
7995    None
7996    None
7997    None
7998    None
7999    None
Length: 8000, dtype: object

# UNIGRAM VECTOR MODEL

In [18]:
#get the vector model in data
unigramVM = []
def getUnigramReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        if parsedText[index] != "br":
            if parsedText[index] in rowVector:
                rowVector[parsedText[index]] +=1
            else:
                rowVector[parsedText[index]] = 1

    if score >= 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    unigramVM.append(rowVector)
    

data.apply(getUnigramReviewVectors, axis = 1)
    
    
    

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7970    None
7971    None
7972    None
7973    None
7974    None
7975    None
7976    None
7977    None
7978    None
7979    None
7980    None
7981    None
7982    None
7983    None
7984    None
7985    None
7986    None
7987    None
7988    None
7989    None
7990    None
7991    None
7992    None
7993    None
7994    None
7995    None
7996    None
7997    None
7998    None
7999    None
Length: 8000, dtype: object

In [19]:
pd.DataFrame(unigramVM).to_csv("UnigramVectorModel.csv", index = False)

# GET TFIDF UNIGRAM MODEL

In [20]:
#make inverted index
invertedIndex = {}
'''
invertedIndex
the => {index : 1}
'''

def makeInvertedIndex(row):
    #get the bigrams
    #for each bigram -> add a count to the inverted index
    #do not double count
    
    #get bigrams
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        eachWord = parsedText[index]
    
        if eachWord not in invertedIndex:
            invertedIndex[eachWord] = [1, {}]

        if row.name not in invertedIndex[eachWord]:
            invertedIndex[eachWord][0] += 1
            invertedIndex[eachWord][1][row.name] = 1

In [21]:
data.apply(makeInvertedIndex, axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7970    None
7971    None
7972    None
7973    None
7974    None
7975    None
7976    None
7977    None
7978    None
7979    None
7980    None
7981    None
7982    None
7983    None
7984    None
7985    None
7986    None
7987    None
7988    None
7989    None
7990    None
7991    None
7992    None
7993    None
7994    None
7995    None
7996    None
7997    None
7998    None
7999    None
Length: 8000, dtype: object

In [22]:
len(invertedIndex[stemmer.stem("good")][1])

2233

In [23]:
TFIDFVM = []
def getTFIDFReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
        if parsedText[index] != "br":
            if parsedText[index] in rowVector:
                rowVector[parsedText[index]] +=1
            else:
                rowVector[parsedText[index]] = 1

    #get individual counts
    for eachWord in rowVector:
        
        TF = rowVector[eachWord]/unigramDict[eachWord]
        
        term = len(invertedIndex[eachWord][1])
        
        
        if term == 0:
            term = 1
            
        IDF = math.log2(numberOfReviews/term)
            
        if IDF < 0:
            print(eachWord)
            
        rowVector[eachWord] = TF*IDF
            
        
    if score >= 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    TFIDFVM.append(rowVector)
    



In [24]:

data.apply(getTFIDFReviewVectors, axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7970    None
7971    None
7972    None
7973    None
7974    None
7975    None
7976    None
7977    None
7978    None
7979    None
7980    None
7981    None
7982    None
7983    None
7984    None
7985    None
7986    None
7987    None
7988    None
7989    None
7990    None
7991    None
7992    None
7993    None
7994    None
7995    None
7996    None
7997    None
7998    None
7999    None
Length: 8000, dtype: object

In [25]:
pd.DataFrame(TFIDFVM).to_csv("UniTFIDFModel.csv", index = False)